#### The final function for the MVP.

___

The goal of this function is to get the user to input an address (rather 'abused' or result of a scam) and get back the whole history of that input... 

1. Fee paid for the tx
2. How much it was sent
3. Where the funds ended up 
4. In how many outputs where the funds splitted
5. History of future movements of the funds 
6. Graph showing the jumps 

___

In [3]:
# importing libraries

import numpy as np
import pandas as pd
from graphviz import Digraph

In [4]:
# importing df created in previous steps

inputs= pd.read_csv('../files/inputs.csv')
outputs= pd.read_csv('../files/outputs.csv')
scams= pd.read_csv('../files/abused_addresses.csv')

In [ ]:
# !pip install gephistreamer

In [ ]:
# need to give it final touches 
# this is kindda clunky on mac !  ????
from gephistreamer import graph
from gephistreamer import streamer

def build_mask(df, values, key):
    masks = [df[key] == v for v in values]
    try:
        mask = np.zeros(masks[0].shape[0])
    except IndexError: # we dont have this address in the input table
        return None, None
    for m in masks:
        mask = (mask | m)

    return masks, mask


def make_dot(address):
    
    node_attr = dict(style="filled", shape="circle", align="left", fontsize="12", ranksep="0.1", height="0.2")
    dot = Digraph(node_attr=node_attr, graph_attr=dict(size="12,12"))
    seen = set()
    
    def add_nodes(addresses):
        
        _, input_mask = build_mask(inputs, addresses, "addresses")
    
        hash_values = inputs[input_mask]["hash"].values.tolist()
        all_masks, output_mask = build_mask(outputs, hash_values, "hash")
        if all_masks is None:
            return dot
        for address, mask in zip(addresses, all_masks):
            if address not in seen:
                dot.node(address, str(address), color='green')
                for edge in outputs[mask]["addresses"].tolist():
                    dot.edge(address, str(edge), constraint='false', color='red')
                
                seen.add(address)

                print(address, 'sent a transaction to the folowing adresses')
                print("\n".join(outputs[mask]["addresses"].tolist()))
                add_nodes(outputs[output_mask]["addresses"].values.tolist())
            
            return dot

            #for address in addresses:
            #    print('\t', address)
            #display(inputs[input_mask])
            #display(outputs[output_mask])
    
    add_nodes(address)
    return dot

g = make_dot(["38Kc1euSQSaXwXT6BMbtzMKor9UX7es8DU"])
g.view()

### Connection should be established properly and nodes should start appearing on gephi software.